Basic orchestration for Frascati.

In the first step, we set the parameters we are interested in.

In [ ]:
roi = 'POLYGON ((11.9 48.0, 11.9 48.5, 11.1 48.1, 11.0 48.0, 11.9 48.0))'
roi_tanzania = 'POLYGON((32. -2., 32. -8., 38. -8., 38. -2., 32. -2.))'
start_time_as_string = '2017-01-01'
end_time_as_string = '2017-01-10'
time_interval = 8
time_interval_unit = 'days' # days or weeks
spatial_resolution_x = 10 # in m
spatial_resolution_y = 10 # in m
variables = ['sm', 'lai', 'cab']

We continue with setting up the Data Access Component. It handles the access to all the data we need.

In [ ]:
from multiply_data_access import DataAccessComponent
data_access_component = DataAccessComponent()
import multiply_workshop_utils.utils as utils
data_stores_file = utils.get_data_stores_file()
data_access_component.read_data_stores(data_stores_file)

First thing we want are the paths to the priors we are interested in.

In [ ]:
priors_sm_dir = data_access_component.get_data_urls(roi, start_time_as_string, 
                                                    end_time_as_string,
                                                    'SoilMoisture')[0]
priors_veg_dir = data_access_component.get_data_urls(roi, start_time_as_string, 
                                                     end_time_as_string,
                                                     'Vegetation')[0]

Now we have everything to create a configuration file: This file serves as input to the components of the platform. In later versions of the platform, it will be possible to simply specify one such file and run the whole process on base of it.

In [ ]:
config = utils.get_config(priors_sm_dir=priors_sm_dir, priors_veg_dir=priors_veg_dir, 
                          roi=roi, start_time=start_time_as_string, 
                          end_time=end_time_as_string, variables=variables, 
                          time_interval=time_interval, 
                          time_interval_unit=time_interval_unit,
                          spatial_resolution_x=spatial_resolution_x,
                          spatial_resolution_y=spatial_resolution_y)
config_file_name = '{}config.yml'.format(utils.get_out_path())
import yaml
yaml.dump(config, open(config_file_name, 'w+'))
from multiply_core.util import AttributeDict
config_as_dict = AttributeDict(**config)

Next, we convert the times to the datetime format so we can propagate time.

In [ ]:
from datetime import datetime
start_time = datetime.strptime(start_time_as_string, "%Y-%m-%d")
end_time = datetime.strptime(end_time_as_string, "%Y-%m-%d")

Now we can get priors for each time step.

In [ ]:
current_time = start_time
prior_file_dicts = []
from multiply_prior_engine import PriorEngine
while current_time < end_time:
    time_string = current_time.strftime("%Y-%m-%d")
    prior_engine = PriorEngine(datestr=time_string, variables=variables, config=(config_file_name))
    prior_file_dicts.append(prior_engine.get_priors())
    current_time = utils.increase_time_step(current_time, time_interval, time_interval_unit)

Now we can also read the EO data we want. Let's start with MODIS A1:

In [ ]:
data_access_component.get_data_urls(roi_tanzania, start_time_as_string, end_time_as_string, 'MCD43A1')

Now MODIS A2 ...

In [ ]:
data_access_component.get_data_urls(roi_tanzania, start_time_as_string, end_time_as_string, 'MCD43A2')

Preprocessed S2 ...

In [ ]:
data_access_component.get_data_urls(roi, start_time_as_string, end_time_as_string, 'S2_L2')

And finally pre-processed S1:

In [ ]:
data_access_component.get_data_urls(roi, start_time_as_string, end_time_as_string, 'S1')